<div class="alert alert-block alert-success">
<h1>Machine Learning MidTermProject<h1>
Amin Abdipour 401133011</h1>
</div>

Run for first time

In [ ]:
# pip install tqdm scipy resampy xgboost librosa
# !wget -r -N -c -np https://physionet.org/files/circor-heart-sound/1.0.3/
# import shutil
# shutil.move('physionet.org', '/content/drive/My Drive/ML')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import os
import os.path as op
import sklearn
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [ ]:
dir = op.join('/content/drive/My Drive/','ML','physionet.org')  # Path to the Data folder

# Attachs


## helper code

In [ ]:
import os

import numpy as np
import scipy.io
import scipy.io.wavfile
import scipy.signal


# Check if a variable is a number or represents a number.
def is_number(x):
    try:
        float(x)
        return True
    except (ValueError, TypeError):
        return False


# Check if a variable is an integer or represents an integer.
def is_integer(x):
    if is_number(x):
        return float(x).is_integer()
    else:
        return False


# Check if a variable is a finite number or represents a finite number.
def is_finite_number(x):
    if is_number(x):
        return np.isfinite(float(x))
    else:
        return False


# Compare normalized strings.
def compare_strings(x, y):
    try:
        return str(x).strip().casefold() == str(y).strip().casefold()
    except AttributeError:  # For Python 2.x compatibility
        return str(x).strip().lower() == str(y).strip().lower()


# Load a WAV file.
def load_wav_file(filename):
    frequency, recording = scipy.io.wavfile.read(filename)
    return recording, frequency


# Load recordings.
def load_recordings(data_folder, data, get_frequencies=False):
    num_locations = get_num_locations(data)
    recording_information = data.split("\n")[1 : num_locations + 1]

    recordings = list()
    frequencies = list()
    for i in range(num_locations):
        entries = recording_information[i].split(" ")
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        recording, frequency = load_wav_file(filename)
        recordings.append(recording)
        frequencies.append(frequency)

    if get_frequencies:
        return recordings, frequencies
    else:
        return recordings


# Get number of recording locations from patient data.
def get_num_locations(data):
    num_locations = None
    for i, l in enumerate(data.split("\n")):
        if i == 0:
            num_locations = int(l.split(" ")[1])
        else:
            break
    return num_locations


# Get recording locations from patient data.
def get_locations(data):
    num_locations = get_num_locations(data)
    locations = list()
    for i, text in enumerate(data.split("\n")):
        entries = text.split(" ")
        if i == 0:
            pass
        elif 1 <= i <= num_locations:
            locations.append(entries[0])
        else:
            break
    return locations


# Sanitize binary values from Challenge outputs.
def sanitize_binary_value(x):
    x = (
        str(x).replace('"', "").replace("'", "").strip()
    )  # Remove any quotes or invisible characters.
    if (is_finite_number(x) and float(x) == 1) or (x in ("True", "true", "T", "t")):
        return 1
    else:
        return 0

## DataProcessing.find_and_load_patient_files

In [ ]:
import os

# from DataProcessing.helper_code import is_integer


# Find patient data files.
def find_patient_files(data_folder):
    # Find patient files.
    filenames = list()
    for f in sorted(os.listdir(data_folder)):
        root, extension = os.path.splitext(f)
        if not root.startswith(".") and extension == ".txt":
            filename = os.path.join(data_folder, f)
            filenames.append(filename)

    # To help with debugging, sort numerically if the filenames are integers.
    roots = [os.path.split(filename)[1][:-4] for filename in filenames]
    if all(is_integer(root) for root in roots):
        filenames = sorted(
            filenames, key=lambda filename: int(os.path.split(filename)[1][:-4])
        )

    return filenames


# Load patient data as a string.
def load_patient_data(filename):
    with open(filename, "r") as f:
        data = f.read()
    return data

## Label extraction

In [ ]:
# Get murmur from patient data.
def get_murmur(data):
    murmur = None
    for text in data.split("\n"):
        if text.startswith("#Murmur:"):
            murmur = text.split(": ")[1]
    if murmur is None:
        raise ValueError(
            "No murmur available. Is your code trying to load labels from the hidden data?"
        )
    return murmur


# Get outcome from patient data.
def get_outcome(data):
    outcome = None
    for text in data.split("\n"):
        if text.startswith("#Outcome:"):
            outcome = text.split(": ")[1]
    if outcome is None:
        raise ValueError(
            "No outcome available. Is your code trying to load labels from the hidden data?"
        )
    return outcome

## DataProcessing.XGBoost_features.metadata

In [ ]:
import numpy as np

# from DataProcessing.helper_code import compare_strings, sanitize_binary_value


# Get age from patient data.
def get_age(data):
    age = None
    for text in data.split("\n"):
        if text.startswith("#Age:"):
            age = text.split(": ")[1].strip()
    return age


# Get sex from patient data.
def get_sex(data):
    sex = None
    for text in data.split("\n"):
        if text.startswith("#Sex:"):
            sex = text.split(": ")[1].strip()
    return sex


# Get height from patient data.
def get_height(data):
    height = None
    for text in data.split("\n"):
        if text.startswith("#Height:"):
            height = float(text.split(": ")[1].strip())
    return height


# Get weight from patient data.
def get_weight(data):
    weight = None
    for text in data.split("\n"):
        if text.startswith("#Weight:"):
            weight = float(text.split(": ")[1].strip())
    return weight


# Get pregnancy status from patient data.
def get_pregnancy_status(data):
    is_pregnant = None
    for text in data.split("\n"):
        if text.startswith("#Pregnancy status:"):
            is_pregnant = bool(sanitize_binary_value(text.split(": ")[1].strip()))
    return is_pregnant


# Extract features from the data.
def get_metadata(data):

    # Extract the age group and replace with the (approximate) number of months
    # for the middle of the age group.
    age_group = get_age(data)

    if compare_strings(age_group, "Neonate"):
        age = 0.5
    elif compare_strings(age_group, "Infant"):
        age = 6
    elif compare_strings(age_group, "Child"):
        age = 6 * 12
    elif compare_strings(age_group, "Adolescent"):
        age = 15 * 12
    elif compare_strings(age_group, "Young Adult"):
        age = 20 * 12
    else:
        age = float("nan")

    # Extract sex. Use one-hot encoding.
    sex = get_sex(data)

    sex_features = np.zeros(2, dtype=int)
    if compare_strings(sex, "Female"):
        sex_features[0] = 1
    elif compare_strings(sex, "Male"):
        sex_features[1] = 1

    # Extract height and weight.
    height = get_height(data)
    weight = get_weight(data)

    # Extract pregnancy status.
    is_pregnant = get_pregnancy_status(data)

    features = np.hstack(([age], sex_features, [height], [weight], [is_pregnant]))

    return np.asarray(features, dtype=np.float32)

## net_feature_extractor.py

In [ ]:
import os
import pickle

import numpy as np
import torch
from tqdm import tqdm

from DataProcessing.find_and_load_patient_files import (
    find_patient_files,
    load_patient_data,
)
from DataProcessing.helper_code import get_num_locations, load_wav_file
from DataProcessing.label_extraction import get_murmur, get_outcome
from HumBugDB.LogMelSpecs.compute_LogMelSpecs import waveform_to_examples


def net_feature_loader(
    recalc_features, train_data_directory, test_data_directory, spectrogram_directory
):

    if not os.path.isdir(spectrogram_directory):
        os.makedirs(spectrogram_directory)
    if recalc_features:
        spectrograms_train, murmurs_train, outcomes_train = calc_patient_features(
            train_data_directory
        )
        repeats = torch.zeros((len(spectrograms_train),))
        for i in range(len(spectrograms_train)):
            for j in range(len(spectrograms_train[i])):
                repeats[i] += len(spectrograms_train[i][j])
        murmurs_train = torch.repeat_interleave(
            torch.Tensor(np.array(murmurs_train)), repeats.to(torch.int32), dim=0
        )
        outcomes_train = torch.repeat_interleave(
            torch.Tensor(np.array(outcomes_train)), repeats.to(torch.int32), dim=0
        )
        spectrograms_train = torch.cat([x for xs in spectrograms_train for x in xs])
        torch.save(
            spectrograms_train, os.path.join(spectrogram_directory, "spec_train")
        )
        torch.save(murmurs_train, os.path.join(spectrogram_directory, "murmurs_train"))
        torch.save(
            outcomes_train, os.path.join(spectrogram_directory, "outcomes_train")
        )

        spectrograms_test, murmurs_test, outcomes_test = calc_patient_features(
            test_data_directory
        )
        repeats = torch.zeros((len(spectrograms_test),))
        for i in range(len(spectrograms_test)):
            for j in range(len(spectrograms_test[i])):
                repeats[i] += len(spectrograms_test[i][j])
        murmurs_test = torch.repeat_interleave(
            torch.Tensor(np.array(murmurs_test)), repeats.to(torch.int32), dim=0
        )
        outcomes_test = torch.repeat_interleave(
            torch.Tensor(np.array(outcomes_test)), repeats.to(torch.int32), dim=0
        )
        spectrograms_test = torch.cat([x for xs in spectrograms_test for x in xs])
        murmurs_test = torch.Tensor(np.array(murmurs_test))
        outcomes_test = torch.Tensor(np.array(outcomes_test))
        torch.save(spectrograms_test, os.path.join(spectrogram_directory, "spec_test"))
        torch.save(murmurs_test, os.path.join(spectrogram_directory, "murmurs_test"))
        torch.save(outcomes_test, os.path.join(spectrogram_directory, "outcomes_test"))
    else:
        spectrograms_train = torch.load(
            os.path.join(spectrogram_directory, "spec_train")
        )
        murmurs_train = torch.load(os.path.join(spectrogram_directory, "murmurs_train"))
        outcomes_train = torch.load(
            os.path.join(spectrogram_directory, "outcomes_train")
        )
        spectrograms_test = torch.load(os.path.join(spectrogram_directory, "spec_test"))
        murmurs_test = torch.load(os.path.join(spectrogram_directory, "murmurs_test"))
        outcomes_test = torch.load(os.path.join(spectrogram_directory, "outcomes_test"))

    return (
        spectrograms_train,
        murmurs_train,
        outcomes_train,
        spectrograms_test,
        murmurs_test,
        outcomes_test,
    )


def patient_feature_loader(recalc_features, data_directory, output_directory):
    if recalc_features == "True":
        spectrograms, murmurs, outcomes = calc_patient_features(data_directory)
        with open(output_directory + "spectrograms", "wb") as fp:
            pickle.dump(spectrograms, fp)
        with open(output_directory + "murmurs", "wb") as fp:
            pickle.dump(murmurs, fp)
        with open(output_directory + "outcomes", "wb") as fp:
            pickle.dump(outcomes, fp)
    else:
        with open(output_directory + "spectrograms", "rb") as fp:
            spectrograms = pickle.load(fp)
        with open(output_directory + "murmurs", "rb") as fp:
            murmurs = pickle.load(fp)
        with open(output_directory + "outcomes", "rb") as fp:
            outcomes = pickle.load(fp)

    return spectrograms, murmurs, outcomes


# Load recordings.
def load_spectrograms(data_directory, data):
    num_locations = get_num_locations(data)
    recording_information = data.split("\n")[1 : num_locations + 1]

    mel_specs = list()
    for i in range(num_locations):
        entries = recording_information[i].split(" ")
        recording_file = entries[2]
        filename = os.path.join(data_directory, recording_file)
        recording, frequency = load_wav_file(filename)
        recording = recording / 32768
        mel_spec = waveform_to_examples(recording, frequency)
        mel_specs.append(mel_spec)
    return mel_specs


def load_spectrograms_yaseen(file_path):

    mel_specs = list()
    recording, frequency = load_wav_file(file_path)
    recording = recording / 32768
    mel_spec = waveform_to_examples(recording, frequency)
    mel_specs.append(mel_spec)

    return mel_specs


def list_wav_files(data_directory):
    wav_files = []
    subfolder_names = []

    for root, dirs, files in os.walk(data_directory):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
                subfolder_names.append(os.path.basename(root))

    return wav_files, subfolder_names


def calc_patient_features(data_directory):

    if "yaseen" in data_directory:
        # Get data and labels
        outcome_classes = [f.name for f in os.scandir(data_directory) if f.is_dir()]
        murmur_classes = outcome_classes
        num_murmur_classes = len(murmur_classes)
        num_outcome_classes = len(outcome_classes)
        patient_files, labels = list_wav_files(data_directory)
        num_patient_files = len(patient_files)
        spectrograms = list()
        murmurs = list()
        outcomes = list()
        for label, file_path in zip(labels, patient_files):
            # Get labels in the right format
            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome =  label
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)
            murmurs = outcomes
            # Spectrograms
            current_spectrograms = load_spectrograms_yaseen(file_path)
            spectrograms.append(current_spectrograms)

    else:
        murmur_classes = ["Present", "Unknown", "Absent"]
        num_murmur_classes = len(murmur_classes)
        outcome_classes = ["Abnormal", "Normal"]
        num_outcome_classes = len(outcome_classes)
        patient_files = find_patient_files(data_directory)
        num_patient_files = len(patient_files)
        spectrograms = list()
        murmurs = list()
        outcomes = list()
        for i in range(num_patient_files):
            # Load the current patient data and recordings.
            current_patient_data = load_patient_data(patient_files[i])
            current_spectrograms = load_spectrograms(data_directory, current_patient_data) # Get spectrograms per patient -> Adjust for Yaseen
            spectrograms.append(current_spectrograms)
            current_murmur = np.zeros(num_murmur_classes, dtype=int)
            murmur = get_murmur(current_patient_data) # -> Adjust for Yaseen
            if murmur in murmur_classes:
                j = murmur_classes.index(murmur)
                current_murmur[j] = 1
            murmurs.append(current_murmur)
            # Outcome
            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome = get_outcome(current_patient_data) # -> Adjust for Yaseen
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)

    return spectrograms, murmurs, outcomes

ModuleNotFoundError: No module named 'DataProcessing'

## Run torch

# Step 1: Data Splits

In [ ]:
import argparse
import os
import shutil

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from tqdm import tqdm

# from DataProcessing.find_and_load_patient_files import (
#     find_patient_files,
#     load_patient_data,
# )
# from DataProcessing.label_extraction import get_murmur, get_outcome
# from DataProcessing.XGBoost_features.metadata import get_metadata


def stratified_test_vali_split(
    stratified_features: list,
    data_directory: str,
    stratified_directory: str,
    test_size: float,
    vali_size: float,
    random_states: list = [42],
    cv: bool = False,
    n_splits: int = 10,
    stratified_cv: bool = False,
):
    # Check if stratified_directory directory exists, otherwise create it.
    if not os.path.exists(stratified_directory):
        os.makedirs(stratified_directory)

    # Get metadata
    patient_files = find_patient_files(data_directory)
    num_patient_files = len(patient_files)
    murmur_classes = ["Present", "Unknown", "Absent"]
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ["Abnormal", "Normal"]
    num_outcome_classes = len(outcome_classes)
    features = list()
    murmurs = list()
    outcomes = list()
    for i in tqdm(range(num_patient_files)):
        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])
        # Extract features.
        current_features = get_metadata(current_patient_data)
        current_features = np.insert(
            current_features, 0, current_patient_data.split(" ")[0]
        )
        current_features = np.insert(
            current_features, 1, current_patient_data.split(" ")[2][:-3]
        )
        features.append(current_features)
        # Extract labels and use one-hot encoding.
        # Murmur
        current_murmur = np.zeros(num_murmur_classes, dtype=int)
        murmur = get_murmur(current_patient_data)
        if murmur in murmur_classes:
            j = murmur_classes.index(murmur)
            current_murmur[j] = 1
        murmurs.append(current_murmur)
        # Outcome
        current_outcome = np.zeros(num_outcome_classes, dtype=int)
        outcome = get_outcome(current_patient_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcomes.append(current_outcome)
    features = np.vstack(features)
    murmurs = np.vstack(murmurs)
    outcomes = np.vstack(outcomes)

    # Combine dataframes
    features_pd = pd.DataFrame(
        features,
        columns=[
            "id",
            "hz",
            "age",
            "female",
            "male",
            "height",
            "weight",
            "is_pregnant",
        ],
    )
    murmurs_pd = pd.DataFrame(murmurs, columns=murmur_classes)
    outcomes_pd = pd.DataFrame(outcomes, columns=outcome_classes)
    complete_pd = pd.concat([features_pd, murmurs_pd, outcomes_pd], axis=1)
    complete_pd["id"] = complete_pd["id"].astype(int).astype(str)
    complete_pd["stratify_column"] = (
        complete_pd[stratified_features].astype(str).agg("-".join, axis=1)
    )

    # Split data
    complete_pd_train_list = list()
    complete_pd_val_list = list()
    complete_pd_test_list = list()
    cnums = list()
    if cv:
        if stratified_cv:
            print("Performing stratified cross-validation")
            skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
            for i, (train_index, test_index) in enumerate(
                skf.split(complete_pd, complete_pd["stratify_column"])
            ):
                cnums.append(f"split_{i}")
                complete_pd_train, complete_pd_test = complete_pd.iloc[train_index], complete_pd.iloc[test_index]
                vali_split = vali_size / (1 - test_size)
                complete_pd_train, complete_pd_val = train_test_split(
                    complete_pd_train,
                    test_size=vali_split,
                    random_state=42,
                    stratify=complete_pd_train["stratify_column"],
                )
                complete_pd_train_list.append(complete_pd_train)
                complete_pd_val_list.append(complete_pd_val)
                complete_pd_test_list.append(complete_pd_test)
        else:
            print("Performing random cross-validation")
            kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
            for i, (train_index, test_index) in enumerate(
                kf.split(complete_pd)
            ):
                cnums.append(f"split_{i}")
                complete_pd_train, complete_pd_test = complete_pd.iloc[train_index], complete_pd.iloc[test_index]
                vali_split = vali_size / (1 - test_size)
                complete_pd_train, complete_pd_val = train_test_split(
                    complete_pd_train,
                    test_size=vali_split,
                    random_state=42,
                )
                complete_pd_train_list.append(complete_pd_train)
                complete_pd_val_list.append(complete_pd_val)
                complete_pd_test_list.append(complete_pd_test)
    else:
        print("Performing statified split")
        for random_state in random_states:
            cnums.append(f"seed_{random_state}")
            complete_pd_train, complete_pd_test = train_test_split(
                complete_pd,
                test_size=test_size,
                random_state=random_state,
                stratify=complete_pd["stratify_column"],
            )
            vali_split = vali_size / (1 - test_size)
            complete_pd_train, complete_pd_val = train_test_split(
                complete_pd_train,
                test_size=vali_split,
                random_state=random_state + 1,
                stratify=complete_pd_train["stratify_column"],
            )
            complete_pd_train_list.append(complete_pd_train)
            complete_pd_val_list.append(complete_pd_val)
            complete_pd_test_list.append(complete_pd_test)

    # Save the files.
    for cnum, complete_pd_train, complete_pd_val, complete_pd_test in zip(
        cnums, complete_pd_train_list, complete_pd_val_list, complete_pd_test_list
    ):
        print(f"Saving split {cnum} with cv {cv} from {len(cnums)} splits...")
        if cv:
            save_folder = os.path.join(stratified_directory, f"cv_{cv}_stratified_{stratified_cv}", cnum)
        else:
            save_folder = os.path.join(stratified_directory, f"cv_{cv}", cnum)
        os.makedirs(os.path.join(save_folder, "train_data"))
        os.makedirs(os.path.join(save_folder, "vali_data"))
        os.makedirs(os.path.join(save_folder, "test_data"))
        with open(os.path.join(save_folder, "split_details.txt"), "w") as text_file:
            text_file.write("This data split is stratified over the following features: \n")
            for feature in stratified_features:
                text_file.write(feature + ", ")
        for f in complete_pd_train["id"]:
            copy_files(
                data_directory,
                f,
                os.path.join(save_folder, "train_data/"),
            )
        for f in complete_pd_val["id"]:
            copy_files(
                data_directory,
                f,
                os.path.join(save_folder, "vali_data/"),
            )
        for f in complete_pd_test["id"]:
            copy_files(
                data_directory,
                f,
                os.path.join(save_folder, "test_data/"),
            )


def copy_files(data_directory: str, ident: str, stratified_directory: str) -> None:
    # Get the list of files in the data folder.
    files = os.listdir(data_directory)
    # Copy all files in data_directory that start with f to stratified_directory
    for f in files:
        if f.startswith(ident):
            _ = shutil.copy(os.path.join(data_directory, f), stratified_directory)




In [ ]:
if __name__ == "__main__":

    print("---------------- Starting data_splits.py to split the data ----------------")

    parser = argparse.ArgumentParser(prog="StratifiedDataSplit")
    parser.add_argument(
        "--data_directory",
        type=str,
        help="The directory containing the data you wish to split.",
        default=op.join(dir,"files/circor-heart-sound/1.0.3/training_data"),
    )
    parser.add_argument(
        "--stratified_directory",
        type=str,
        help="The directory to store the split data.",
        default=dir,
    )
    parser.add_argument(
        "--vali_size", type=float, default=0.16, help="The size of the test split."
    )
    parser.add_argument(
        "--test_size", type=float, default=0.2, help="The size of the test split."
    )
    parser.add_argument(
        "--cv", type=bool, default=False, help="Whether to run cv."
    )
    parser.add_argument(
        "--stratified_cv", type=bool, default=False, help="Whether to run cv."
    )
    args = parser.parse_known_args()[0] # Allow unrecognized arguments

    # args = parser.parse_args()

    stratified_features = ["Normal", "Abnormal", "Absent", "Present", "Unknown"]
    print('hello')
    # Create the test split.
    stratified_test_vali_split(stratified_features, **vars(args))
    print('hi')

---------------- Starting data_splits.py to split the data ----------------
hello


100%|██████████| 942/942 [03:10<00:00,  4.95it/s] 


Performing statified split
Saving split seed_42 with cv False from 1 splits...


FileExistsError: [Errno 17] File exists: '/content/drive/My Drive/ML/physionet.org/cv_False/seed_42/train_data'

# Step 2: Train ResNet

In [ ]:
# Config file containing all of the model hyperparameters.

# ResNet parameters
dropout = 0.3
# Learning rate
lr = 3e-5
max_overrun = 10
epochs = 200
batch_size = 128
pretrained = True
# Number of classes for multi class classification
n_classes = 3

# Parameters for generating the log mel spectrograms used during training.
# Architectural constants.
NUM_BANDS = 64  # Frequency bands in input mel-spectrogram patch.
EMBEDDING_SIZE = 128  # Size of embedding layer.
# Hyperparameters used in feature and example generation.
SAMPLE_RATE = 4000 # 8000 for Yaseen, else 4000
STFT_WINDOW_LENGTH_SECONDS = 0.025
STFT_HOP_LENGTH_SECONDS = 0.010
NUM_MEL_BINS = NUM_BANDS
MEL_MIN_HZ = 10
MEL_MAX_HZ = 2000
LOG_OFFSET = 0.01  # Offset used for stabilized log of input mel-spectrogram.
EXAMPLE_WINDOW_SECONDS = 4.0 # 0.5 for Yaseen, else 4.0
EXAMPLE_HOP_SECONDS = 1.0 # 0.01 for Yaseen, else 1.0

In [ ]:
import os

import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from Config import hyperparameters
from HumBugDB.ResNetDropoutSource import resnet50dropout
from HumBugDB.ResNetSource import resnet50


class ResnetFull(nn.Module):
    def __init__(self):
        super(ResnetFull, self).__init__()
        self.resnet = resnet50(pretrained=hyperparameters.pretrained)
        self.n_channels = 3
        # Remove final linear layer
        self.resnet = nn.Sequential(*(list(self.resnet.children())[:-1]))
        self.fc1 = nn.Linear(2048, 1)

    def forward(self, x):
        x = self.resnet(x).squeeze()
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x


class ResnetDropoutFull(nn.Module):
    def __init__(self, dropout=0.2, bayesian=True):
        super(ResnetDropoutFull, self).__init__()
        self.dropout = dropout
        self.resnet = resnet50dropout(
            pretrained=hyperparameters.pretrained, dropout_p=self.dropout, bayesian=bayesian
        )
        self.n_channels = 3
        # Remove final linear layer
        self.resnet = nn.Sequential(*(list(self.resnet.children())[:-1]))
        self.fc1 = nn.Linear(2048, 1)
        self.bayesian = bayesian

    def forward(self, x):
        if self.bayesian == True:
            training = True
        else:
            training = self.training
        x = self.resnet(x).squeeze()
        x = self.fc1(F.dropout(x, p=self.dropout, training=training))
        x = torch.sigmoid(x)
        return x


def make_weights_for_balanced_classes(images, nclasses):
    count = [0] * nclasses
    for item in images:
        count[torch.argmax(item[1])] += 1
    weight_per_class = [0.0] * nclasses
    N = float(sum(count))
    for i in range(nclasses):
        weight_per_class[i] = N / float(count[i])
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[torch.argmax(val[1])]
    return weight


def build_dataloader(
    x_train, y_train, x_val=None, y_val=None, shuffle=True, sampler=None
):
    x_train = x_train.clone().detach().float() #torch.tensor(x_train).float()
    y_train = y_train.clone().detach().float() #torch.tensor(y_train).float()
    train_dataset = TensorDataset(x_train, y_train)
    if sampler is None:
        train_loader = DataLoader(
            train_dataset, batch_size=hyperparameters.batch_size, shuffle=shuffle
        )
    else:
        weights = make_weights_for_balanced_classes(train_dataset, 2)
        weights = torch.DoubleTensor(weights)
        sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
        train_loader = DataLoader(
            train_dataset, batch_size=hyperparameters.batch_size, sampler=sampler
        )

    if x_val is not None:
        x_val = x_val.clone().detach().float() #torch.tensor(x_val).float()
        y_val = y_val.clone().detach().float() #torch.tensor(y_val).float()
        val_dataset = TensorDataset(x_val, y_val)
        val_loader = DataLoader(
            val_dataset, batch_size=hyperparameters.batch_size, shuffle=shuffle
        )

        return train_loader, val_loader
    return train_loader


def train_model(
    x_train,
    y_train,
    clas_weight=None,
    x_val=None,
    y_val=None,
    model=ResnetDropoutFull(),
    model_name="test",
    model_dir="models",
    sampler=None,
):

    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)

    if x_val is not None:
        train_loader, val_loader = build_dataloader(
            x_train, y_train, x_val, y_val, sampler=sampler
        )

    else:
        train_loader = build_dataloader(x_train, y_train, sampler=sampler)

    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

    if torch.cuda.device_count() > 1:
        print("Using data parallel")
        model = nn.DataParallel(
            model, device_ids=list(range(torch.cuda.device_count()))
        )

    model = model.to(device)
    criterion = nn.BCELoss()
    optimiser = optim.Adam(model.parameters(), lr=hyperparameters.lr)

    all_train_loss = []
    all_train_metric = []
    all_val_loss = []
    all_val_metric = []
    best_val_acc = -np.inf

    best_train_acc = -np.inf
    e_saved = None
    output_string_to_save = ""
    overrun_counter = 0
    for e in range(hyperparameters.epochs):
        start_time = time.time()
        train_loss = 0.0
        model.train()

        all_y = []
        all_y_pred = []
        for batch_i, inputs in enumerate(train_loader):

            x = inputs[:-1][0].repeat(1, 3, 1, 1)
            y = torch.argmax(inputs[1], dim=1, keepdim=True).float()

            optimiser.zero_grad()
            y_pred = model(x)
            if clas_weight is not None:
                criterion.weight = (clas_weight[1] - clas_weight[0]) * y + clas_weight[
                    0
                ]
                loss = criterion.forward(y_pred, y)
            else:
                loss = criterion(y_pred, y)

            loss.backward()
            optimiser.step()

            train_loss += loss.item()
            all_y.append(y.cpu().detach())
            all_y_pred.append(y_pred.cpu().detach())

            del x
            del y

        all_train_loss.append(train_loss / len(train_loader))

        all_y = torch.cat(all_y)
        all_y_pred = torch.cat(all_y_pred)
        train_metric = balanced_accuracy_score(
            all_y.numpy(), (all_y_pred.numpy() > 0.5).astype(float)
        )
        all_train_metric.append(train_metric)

        if x_val is not None:
            val_loss, val_metric = test_model(
                model, val_loader, clas_weight, criterion, device=device
            )
            all_val_loss.append(val_loss)
            all_val_metric.append(val_metric)

            acc_metric = val_metric
            best_acc_metric = best_val_acc
        else:
            acc_metric = train_metric
            best_acc_metric = best_train_acc
        if acc_metric > best_acc_metric:

            checkpoint_name = f"model_{model_name}.pth"
            e_saved = e
            torch.save(
                model.state_dict(),
                os.path.join(model_dir, checkpoint_name),
            )
            print(
                "Saving model to:",
                os.path.join(model_dir, checkpoint_name),
            )
            best_train_acc = train_metric
            if x_val is not None:
                best_val_acc = val_metric
            overrun_counter = -1

        overrun_counter += 1
        if x_val is not None:
            output_string = (
                "Epoch: %d, Train Loss: %.8f, Train Acc: %.8f, Val Loss: %.8f, "
                "Val Acc: %.8f, overrun_counter %i"
                % (
                    e,
                    train_loss / len(train_loader),
                    train_metric,
                    val_loss,
                    val_metric,
                    overrun_counter,
                )
            )
        else:
            output_string = (
                "Epoch: %d, Train Loss: %.8f, Train Acc: %.8f, overrun_counter %i"
                % (e, train_loss / len(train_loader), train_metric, overrun_counter)
            )
        print(output_string)
        output_string_to_save += output_string + "\n"
        print(f"Training epoch {e} took {round((time.time()-start_time)/60,4)} min.")
        if overrun_counter > hyperparameters.max_overrun:
            break

    if e_saved is not None:
        checkpoint_name = f"model_{model_name}_final.pth"
        torch.save(
            model.state_dict(),
            os.path.join(model_dir, checkpoint_name),
        )
        print(
            "Saving model to:",
            os.path.join(model_dir, checkpoint_name),
        )

    # Save output string
    output_string_to_save += f"Best epoch: {e_saved}\n"
    with open(os.path.join(model_dir, f"output_{model_name}.txt"), "w") as f:
        f.write(output_string_to_save)

    return model


def test_model(model, test_loader, clas_weight, criterion, device=None):
    with torch.no_grad():
        if device is None:
            torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

        test_loss = 0.0
        model.eval()

        all_y = []
        all_y_pred = []
        counter = 1
        for inputs in test_loader:

            x = inputs[:-1][0].repeat(1, 3, 1, 1)
            y = torch.argmax(inputs[1], dim=1, keepdim=True).float()

            if len(x) == 1:
                x = x[0]

            y_pred = model(x)

            if clas_weight is not None:
                criterion.weight = (clas_weight[1] - clas_weight[0]) * y + clas_weight[
                    0
                ]
                loss = criterion.forward(y_pred, y)
            else:
                loss = criterion(y_pred, y)
            test_loss += loss.item()
            all_y.append(y.cpu().detach())
            all_y_pred.append(y_pred.cpu().detach())

            del x
            del y
            del y_pred

            counter += 1

        all_y = torch.cat(all_y)
        all_y_pred = torch.cat(all_y_pred)
        test_metric = balanced_accuracy_score(
            all_y.numpy(), (all_y_pred.numpy() > 0.5).astype(float)
        )
        test_loss = test_loss / len(test_loader)

    return test_loss, test_metric


def load_model(filepath, model=ResnetDropoutFull()):

    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

    if torch.cuda.device_count() > 1:
        print("Using data parallel")
        model = nn.DataParallel(
            model, device_ids=list(range(torch.cuda.device_count()))
        )
    model = model.to(device)

    if torch.cuda.is_available():
        map_location = lambda storage, loc: storage.cuda()
    elif torch.backends.mps.is_available():
        map_location = lambda storage, loc: storage.mps()
    else:
        map_location = torch.device("cpu")
    model.load_state_dict(torch.load(filepath, map_location=map_location))

    return model

In [ ]:
import argparse

import torch

# from Config import hyperparameters
# from DataProcessing.net_feature_extractor import net_feature_loader
from HumBugDB.runTorch import ResnetDropoutFull as ResnetDropoutBinary
from HumBugDB.runTorch import ResnetFull as ResnetBinary
from HumBugDB.runTorch import train_model as train_model_binary
from HumBugDB.runTorchMultiClass import ResnetDropoutFull as ResnetDropoutMulti
from HumBugDB.runTorchMultiClass import ResnetFull as ResnetMulti
from HumBugDB.runTorchMultiClass import train_model as train_model_multi


def create_model(model_name, num_classes, bayesian):
    if model_name == "resnet50":
        print("Running resnet without dropout")
        if num_classes == 2:
            model = ResnetBinary()
            training = train_model_binary
        else:
            model = ResnetMulti(num_classes)
            training = train_model_multi
    elif model_name == "resnet50dropout":
        print(f"Creating dropout model with bayesian: {bayesian}")
        if num_classes == 2:
            model = ResnetDropoutBinary(dropout=hyperparameters.dropout, bayesian=bayesian)
            training = train_model_binary
        else:
            model = ResnetDropoutMulti(n_classes=num_classes, bayesian=bayesian)
            training = train_model_multi
    else:
        raise NotImplementedError("Only implemented resnet50 and resnet50dropout")

    return model, training


def run_model_training(
    recalc_features,
    train_data_directory,
    vali_data_directory,
    spectrogram_directory,
    model_name,
    model_label,
    model_dir,
    classes_name,
    bayesian,
    weights,
):

    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
    print("Using device:", device)

    (
        spectrograms_train,
        murmurs_train,
        outcomes_train,
        spectrograms_test,
        murmurs_test,
        outcomes_test,
    ) = net_feature_loader(
        recalc_features,
        train_data_directory,
        vali_data_directory,
        spectrogram_directory,
    )
    print("Data loaded")

    X_train = spectrograms_train.to(device)
    X_test = spectrograms_test.to(device)
    if classes_name == "murmur":
        y_train = murmurs_train.to(device)
        y_test = murmurs_test.to(device)
        model, training = create_model(model_name, 3, bayesian)
        training(
            X_train,
            y_train,
            clas_weight=weights,
            x_val=X_test,
            y_val=y_test,
            model=model,
            model_name=model_label,
            model_dir=model_dir,
        )
    elif classes_name == "outcome_binary":
        y_train = outcomes_train.to(device)
        y_test = outcomes_test.to(device)
        model, training = create_model(model_name, 2, bayesian)
        training(
            X_train,
            y_train,
            clas_weight=weights,
            x_val=X_test,
            y_val=y_test,
            model=model,
            model_name=model_label,
            model_dir=model_dir,
        )
    elif classes_name == "murmur_binary":
        knowledge_train = torch.zeros((murmurs_train.shape[0], 2))
        for i in range(len(murmurs_train)):
            if (
                torch.argmax(murmurs_train[i]) == 0
                or torch.argmax(murmurs_train[i]) == 1
            ):
                knowledge_train[i, 0] = 1
            else:
                knowledge_train[i, 1] = 1
        knowledge_test = torch.zeros((murmurs_test.shape[0], 2))
        for i in range(len(murmurs_test)):
            if torch.argmax(murmurs_test[i]) == 0 or torch.argmax(murmurs_test[i]) == 1:
                knowledge_test[i, 0] = 1
            else:
                knowledge_test[i, 1] = 1
        y_train = knowledge_train.to(device)
        y_test = knowledge_test.to(device)
        model, training = create_model(model_name, 2, bayesian)
        training(
            X_train,
            y_train,
            clas_weight=weights,
            x_val=X_test,
            y_val=y_test,
            model=model,
            model_name=model_label,
            model_dir=model_dir,
        )
    elif classes_name == "binary_present":
        knowledge_train = torch.zeros((murmurs_train.shape[0], 2))
        for i in range(len(murmurs_train)):
            if (
                torch.argmax(murmurs_train[i]) == 1
                or torch.argmax(murmurs_train[i]) == 2
            ):
                knowledge_train[i, 1] = 1
            else:
                knowledge_train[i, 0] = 1
        knowledge_test = torch.zeros((murmurs_test.shape[0], 2))
        for i in range(len(murmurs_test)):
            if torch.argmax(murmurs_test[i]) == 1 or torch.argmax(murmurs_test[i]) == 2:
                knowledge_test[i, 1] = 1
            else:
                knowledge_test[i, 0] = 1
        y_train = knowledge_train.to(device)
        y_test = knowledge_test.to(device)
        model, training = create_model(model_name, 2, bayesian)
        training(
            X_train,
            y_train,
            clas_weight=weights,
            x_val=X_test,
            y_val=y_test,
            model=model,
            model_name=model_label,
            model_dir=model_dir,
        )
    elif classes_name == "binary_unknown":
        knowledge_train = torch.zeros((murmurs_train.shape[0], 2))
        for i in range(len(murmurs_train)):
            if (
                torch.argmax(murmurs_train[i]) == 0
                or torch.argmax(murmurs_train[i]) == 2
            ):
                knowledge_train[i, 1] = 1
            else:
                knowledge_train[i, 0] = 1
        knowledge_test = torch.zeros((murmurs_test.shape[0], 2))
        for i in range(len(murmurs_test)):
            if torch.argmax(murmurs_test[i]) == 0 or torch.argmax(murmurs_test[i]) == 2:
                knowledge_test[i, 1] = 1
            else:
                knowledge_test[i, 0] = 1
        y_train = knowledge_train.to(device)
        y_test = knowledge_test.to(device)
        model, training = create_model(model_name, 2, bayesian)
        training(
            X_train,
            y_train,
            clas_weight=weights,
            x_val=X_test,
            y_val=y_test,
            model=model,
            model_name=model_label,
            model_dir=model_dir,
            sampler=True,
        )
    else:
        raise ValueError("classes_name must be one of outcome, murmur or knowledge.")


if __name__ == "__main__":

    parser = argparse.ArgumentParser(prog="TrainResNet")
    parser.add_argument(
        "--recalc_features",
        action="store_true",
        help="Whether or not to recalculate the log mel spectrograms used as "
        "input to the ResNet.",
    )
    parser.add_argument(
        "--no-recalc_features", dest="recalc_features", action="store_false"
    )
    parser.set_defaults(recalc_features=True)
    parser.add_argument(
        "--train_data_directory",
        type=str,
        help="The directory of the training data.",
        default="data/stratified_data/train_data",
    )
    parser.add_argument(
        "--vali_data_directory",
        type=str,
        help="The directory of the validation data.",
        default="data/stratified_data/vali_data",
    )
    parser.add_argument(
        "--spectrogram_directory",
        type=str,
        help="The directory in which to save the spectrogram training data.",
        default="data/spectrograms",
    )
    parser.add_argument(
        "--model_name",
        type=str,
        help="The ResNet to train. Current options are resnet50 or resnet50dropout.",
        choices=["resnet50", "resnet50dropout"],
        default="resnet50dropout",
    )
    parser.add_argument(
        "--model_label",
        type=str,
        help="The label to use when saving the model.",
        default="ResNetDropout",
    )
    parser.add_argument(
        "--model_dir",
        type=str,
        help="The directory to use when saving the model.",
        default="data/models",
    )
    parser.add_argument(
        "--classes_name",
        type=str,
        help="The name of the classes to train the model on.",
        choices=["murmur", "outcome_binary", "murmur_binary", "binary_present", "binary_unknown"],
        default="murmur",
    )
    parser.add_argument(
        '--disable-bayesian',
        dest='bayesian',
        action='store_false',
        default=True,
        help='Disable Bayesian features (default: Bayesian is enabled)'
    )
    parser.add_argument(
        "--weights_str",
        type=str,
        help="String containing the class weights for a weighted loss function, "
        "e.g.5,3,1.",
        default=None,
    )

    args = parser.parse_args()

    weights = None
    if args.weights_str:
        weights = [int(x) for x in args.weights_str.split(",")]
    vars(args).popitem()

    print("---------------- Starting train_resnet.py for training ----------------")
    print(f"---------------- Using data from {args.train_data_directory}")

    run_model_training(**vars(args), weights=weights)

# Step 3: dbres

In [ ]:
import argparse
import os

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from DataProcessing.find_and_load_patient_files import (
    find_patient_files,
    load_patient_data,
)
from DataProcessing.helper_code import get_num_locations, load_recordings, load_wav_file
from DataProcessing.net_feature_extractor import load_spectrograms_yaseen, load_spectrograms_yaseen
from HumBugDB.LogMelSpecs.compute_LogMelSpecs import waveform_to_examples
from HumBugDB.runTorch import load_model
from ModelEvaluation.evaluate_model import evaluate_model
from train_resnet import create_model

from Config import hyperparameters


def list_wav_files(data_directory):
    wav_files = []
    subfolder_names = []

    for root, dirs, files in os.walk(data_directory):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
                subfolder_names.append(os.path.basename(root))

    return wav_files, subfolder_names


def get_binary_spectrogram_outputs(
    spectrograms,
    model_binary_present,
    model_binary_unknown,
    model_binary
):
    if (model_binary_present is not None) and (model_binary_unknown is not None):
        model_outputs_unknown = []
        model_outputs_present = []
        unknown_probabilities = []
        present_probabilities = []
        for spectrogram in spectrograms:
            output_present = (
                calc_patient_output(model_binary_present, [spectrogram], repeats=30)
                .cpu()
                .numpy()
            )
            output_unknown = (
                calc_patient_output(model_binary_unknown, [spectrogram], repeats=30)
                .cpu()
                .numpy()
            )
            model_outputs_present.append(output_present)
            model_outputs_unknown.append(output_unknown)
            present_probabilities.append(
                np.array([1 - output_present[0], output_present[0]])
            )
            unknown_probabilities.append(
                np.array([1 - output_unknown[0], output_unknown[0]])
            )
        present_probability = np.mean(np.array(present_probabilities), axis=0)
        unknown_probability = np.mean(np.array(unknown_probabilities), axis=0)
        outputs = []
        idx_unknown = (np.mean(np.array(model_outputs_unknown)) > 0.5).astype(float)
        idx_present = (np.mean(np.array(model_outputs_present)) > 0.5).astype(float)
        if idx_present == 0:
            outputs.append(np.array([1, 0, 0]))
        elif idx_unknown == 0:
            outputs.append(np.array([0, 1, 0]))
        else:
            outputs.append(np.array([0, 0, 1]))

        probabilities = [
            present_probability[0],
            present_probability[1] * unknown_probability[0],
            present_probability[1] * unknown_probability[1],
        ]
    elif model_binary is not None:
        model_outputs = []
        probabilities = []
        for spectrogram in spectrograms:
            output = (
                calc_patient_output(model_binary, [spectrogram], repeats=30)
                .cpu()
                .numpy()
            )
            model_outputs.append(output)
            probabilities.append(np.array([1 - output[0], output[0]]))
        probability = np.mean(np.array(probabilities), axis=0)
        outputs = []
        idx = (np.mean(np.array(model_outputs)) > 0.5).astype(float)
        if idx == 0:
            outputs.append(np.array([1, 0]))
        else:
            outputs.append(np.array([0, 1]))
        probabilities = [probability[0], probability[1]]

    return outputs[0].tolist(), probabilities


def calc_patient_output(model, recording_spectrograms, repeats):
    model.eval()
    outputs = []
    for location in recording_spectrograms:
        input = location.repeat(1, 3, 1, 1)
        device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
        input = input.to(device)
        model_out = []
        for _ in range(repeats):
            out = model(input)
            out = out.cpu().detach().unsqueeze(2)
            model_out.append(out)
        model_out = torch.mean(torch.cat(model_out, dim=2), dim=2)
        outputs.append(torch.mean(model_out, axis=0).unsqueeze(dim=0))
    output = torch.mean(torch.cat(outputs), axis=0).detach()
    return output


def calculate_dbres_output(
    model_name,
    recalc_output,
    data_directory,
    output_directory,
    model_binary_pth,
    model_binary_present_pth,
    model_binary_unknown_pth,
    recordings_file: str = "",
    bayesian: bool = True
):

    if recalc_output:

        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        # Get model
        model_binary_present = create_model(model_name, 2, bayesian)
        model_binary_unknown = create_model(model_name, 2, bayesian)
        model_binary = create_model(model_name, 2, bayesian)

        # Load model
        if (model_binary_present_pth is not None) and (model_binary_unknown_pth is not None):
            print("Loading multiclass model")
            model_binary_present = load_model(
                model_binary_present_pth, model=model_binary_present[0]
            )
            model_binary_unknown = load_model(
                model_binary_unknown_pth, model=model_binary_unknown[0]
            )
            model_binary = None
        elif model_binary_pth is not None:
            print("Loading binary model")
            model_binary = load_model(model_binary_pth, model=model_binary[0])
            model_binary_present = None
            model_binary_unknown = None
        else:
            raise Exception("No model was provided.")

        # Get data
        murmur_probabilities = list()
        murmur_outputs = list()
        labels = None
        if len(recordings_file) > 0:
            patient_files = pd.read_csv(recordings_file)
        else:
            if "yaseen" in data_directory:
                outcome_classes = [f.name for f in os.scandir(data_directory) if f.is_dir()]
                murmur_classes = outcome_classes
                patient_files, labels = list_wav_files(data_directory)
            else:
                patient_files = find_patient_files(data_directory)

        # Get count of patient files
        num_patient_files = len(patient_files)
        if num_patient_files == 0:
            print(f"No data was provided in {data_directory} for recordings_file {recordings_file}.")
            raise Exception("No data was provided.")

        # Get spectrograms and predictions
        for i in tqdm(range(num_patient_files)):
            if len(recordings_file) > 0:
                current_patient_data = patient_files.iloc[i]
                current_recordings = list()
                recording, frequency = load_wav_file(patient_files["path"].iloc[i])
                current_recordings.append(recording)
                sample_rate = frequency
                num_locations = 1
            else:
                if "yaseen" in data_directory:
                    pass
                else:
                    sample_rate=hyperparameters.SAMPLE_RATE
                    current_patient_data = load_patient_data(patient_files[i])
                    current_recordings = load_recordings(data_directory, current_patient_data)
                    num_locations = get_num_locations(current_patient_data)

            # Get spectrograms
            if "yaseen" in data_directory:
                spectrograms = load_spectrograms_yaseen(patient_files[i])
            else:
                current_recordings = [r / 32768 for r in current_recordings]
                spectrograms = list()
                for j in range(num_locations):
                    mel_spec = waveform_to_examples(
                        data=current_recordings[j], sample_rate=sample_rate
                    )
                    spectrograms.append(mel_spec)

            # Get predictions
            murmur_output, murmur_probability = get_binary_spectrogram_outputs(
                spectrograms, model_binary_present, model_binary_unknown, model_binary
            )
            murmur_probabilities.append(murmur_probability)
            murmur_outputs.append(murmur_output)

        # Store
        murmur_probabilities = np.vstack(murmur_probabilities)
        np.save(
            os.path.join(output_directory, "probabilities.npy"),
            murmur_probabilities,
        )
        murmur_outputs = np.vstack(murmur_outputs)
        np.save(os.path.join(output_directory, "outputs.npy"), murmur_outputs)
    else:
        murmur_probabilities = np.load(
            os.path.join(output_directory, "probabilities.npy")
        )
        murmur_outputs = np.load(os.path.join(output_directory, "outputs.npy"))

    return murmur_probabilities, murmur_outputs, labels


def calculate_dbres_scores(
    model_name,
    recalc_output,
    data_directory,
    output_directory,
    model_binary_pth,
    model_binary_present_pth,
    model_binary_unknown_pth,
    recordings_file: str = "",
    bayesian: bool = True
):

    probabilities, outputs, labels = calculate_dbres_output(
        model_name,
        recalc_output,
        data_directory,
        output_directory,
        model_binary_pth,
        model_binary_present_pth,
        model_binary_unknown_pth,
        recordings_file,
        bayesian
    )

    if (model_binary_present_pth is not None) and (model_binary_unknown_pth is not None):
        model_type = "murmur"
    elif model_binary_pth is not None:
        if ("MurmurBinary" in model_binary_pth) or ("Murmur_Binary" in model_binary_pth):
            model_type = "murmur_binary"
        elif ("OutcomeBinary" in model_binary_pth) or ("Outcome_Binary" in model_binary_pth):
            model_type = "outcome_binary"
        else:
            raise Exception("No binary murmur or outcome model was provided.")
    else:
        raise Exception("No model was provided.")

    print(f"--- Evaluating {model_type} model ---")
    if "yaseen" in data_directory:
        scores = evaluate_model(data_directory, probabilities, outputs, model_type, recordings_file = recordings_file, output_directory = output_directory, true_labels = labels)
    else:
        scores = evaluate_model(data_directory, probabilities, outputs, model_type, recordings_file = recordings_file, output_directory = output_directory)

    print("--- DBRes scores ---")
    print(f"{scores}")
    with open(os.path.join(output_directory, "DBRes_score.npy"), "w") as text_file:
        text_file.write(scores)

    if model_type == "murmur":
        print(f"--- Evaluating {model_type} model as binary ---")
        # Combine element at position 0 and 1 to get binary output, but keep position 2
        outputs_binary = np.vstack(
            [np.logical_or(outputs[:, 0], outputs[:, 1]), outputs[:, 2]]
        ).T
        probabilities_binary = np.vstack(
            [np.max(probabilities[:, :2], axis=1), probabilities[:, 2]]
        ).T
        scores_binary = evaluate_model(
            data_directory, probabilities_binary, outputs_binary, "murmur_binary", recordings_file = recordings_file, output_directory = output_directory
        )
        print("--- DBRes scores binary ---")
        print(f"{scores_binary}")
        with open(os.path.join(output_directory, "DBRes_score_binary.npy"), "w") as text_file:
            text_file.write(scores_binary)

    return scores


if __name__ == "__main__":

    parser = argparse.ArgumentParser(prog="DBRes")
    parser.add_argument(
        "--model_name",
        type=str,
        help="The ResNet to train. Current options are resnet50 or resnet50dropout.",
        choices=["resnet50", "resnet50dropout"],
        default="resnet50dropout",
    )
    parser.add_argument(
        "--recalc_output",
        action="store_true",
        help="Whether or not to recalculate the output from DBRes.",
    )
    parser.add_argument(
        "--no-recalc_output", dest="recalc_output", action="store_false"
    )
    parser.set_defaults(recalc_output=True)
    parser.add_argument(
        "--data_directory",
        type=str,
        help="The directory of the data.",
        default="data/stratified_data/test_data",
    )
    parser.add_argument(
        "--output_directory",
        type=str,
        help="The directory in which to save DBRes's output.",
        default="data/dbres_outputs",
    )
    parser.add_argument(
        "--model_binary_pth",
        type=str,
        help="The path of binary ResNet trained to classify present vs not present.",
        default=None,
    )
    parser.add_argument(
        "--model_binary_present_pth",
        type=str,
        help="The path of binary ResNet trained to classify present vs not present.",
        default=None,
    )
    parser.add_argument(
        "--model_binary_unknown_pth",
        type=str,
        help="The path of binary ResNet trained to classify unknown vs not unknown.",
        default=None,
    )
    parser.add_argument(
        "--recordings_file",
        type=str,
        help="The path to a recordings file.",
        default="",
    )
    parser.add_argument(
        '--disable-bayesian',
        dest='bayesian',
        action='store_false',
        default=True,
        help='Disable Bayesian features (default: Bayesian is enabled)'
    )

    args = parser.parse_args()

    print("---------------- Starting dbres.py for predictions and evaluations ----------------")
    if len(args.recordings_file) > 0:
        print(f"---------------- Using data from {args.recordings_file}")
    else:
        print(f"---------------- Using data from {args.data_directory}")

    scores = calculate_dbres_scores(**vars(args))

# Step 4: xgboost_integration

In [ ]:
import argparse
import os
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from tqdm import tqdm

from DataProcessing.find_and_load_patient_files import (
    find_patient_files,
    load_patient_data,
)
from DataProcessing.helper_code import load_recordings
from DataProcessing.label_extraction import get_murmur, get_outcome
from DataProcessing.XGBoost_features.extract_all_features import extract_all_features
from dbres import calculate_dbres_output
from ModelEvaluation.evaluate_model import evaluate_model


def get_murmurs_features(
    model_name,
    data_directory,
    recalc_dbres_output,
    dbres_output_directory,
    model_binary_pth,
    model_binary_present_pth,
    model_binary_unknown_pth,
    model_type,
    bayesian
):
    patient_files = find_patient_files(data_directory)
    num_patient_files = len(patient_files)

    # Extract the features and labels.
    if model_type == "murmur":
        label_classes = ["Present", "Unknown", "Absent"]
    elif model_type == "murmur_binary":
        label_classes = ["Present", "Absent"]
    elif model_type == "outcome_binary":
        label_classes = ["Abnormal", "Normal"]
    num_label_classes = len(label_classes)

    features = list()
    labels = list()
    for i in range(num_patient_files):

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])
        current_recordings = load_recordings(data_directory, current_patient_data)
        current_recordings = [r / 32768 for r in current_recordings]

        # Extract features.
        metadata_features, audio_features = extract_all_features(
            current_patient_data, current_recordings
        )
        audio_features_reshaped = audio_features.reshape(1, -1)[0]
        current_features = np.hstack((metadata_features, audio_features_reshaped))
        features.append(current_features)

        # Extract labels and use one-hot encoding.
        current_label = np.zeros(num_label_classes, dtype=int)
        if model_type == "murmur":
            label = get_murmur(current_patient_data)
        elif model_type == "murmur_binary":
            label = get_murmur(current_patient_data)
            if label == "Unknown":
                label = "Present"
        elif model_type == "outcome_binary":
            label = get_outcome(current_patient_data)
        if label in label_classes:
            j = label_classes.index(label)
            current_label[j] = 1
        labels.append(current_label)

    features = np.vstack(features)
    labels = np.vstack(labels)

    imputer = SimpleImputer().fit(features)
    features = imputer.transform(features)

    _, spectrogram_outputs = calculate_dbres_output(
        model_name,
        recalc_dbres_output,
        data_directory,
        dbres_output_directory,
        model_binary_pth,
        model_binary_present_pth,
        model_binary_unknown_pth,
        bayesian
    )

    features_combined = np.vstack(
        [np.concatenate((f, s)) for f, s in zip(features, spectrogram_outputs)]
    )

    return labels, features_combined


def train_xgboost_integration(
    model_name,
    train_data_directory,
    dbres_output_directory,
    model_binary_pth,
    model_binary_present_pth,
    model_binary_unknown_pth,
    model_type,
    use_weights=False,
    bayesian=True,
):

    if os.path.exists(dbres_output_directory):
        recalculated_dbres_output = False
    else:
        recalculated_dbres_output = True
        print(f"DBRes output directory {dbres_output_directory} does not exist. Recalculating DBRes output.")

    murmurs, features_combined = get_murmurs_features(
        model_name,
        train_data_directory,
        recalculated_dbres_output,
        dbres_output_directory,
        model_binary_pth,
        model_binary_present_pth,
        model_binary_unknown_pth,
        model_type=model_type,
        bayesian=bayesian
    )

    if use_weights:
        w_pos = 5
        print(f"Using postivie class sample weight {w_pos}.")
        sample_weights = np.ones(len(murmurs))
        for i in range(len(murmurs)):
            if murmurs[i][0] == 1:
                sample_weights[i] = w_pos
    else:
        print("Not using sample weights.")
        sample_weights = None

    murmur_classifier = xgb.XGBClassifier()
    murmur_classifier.fit(features_combined, murmurs, sample_weight = sample_weights)

    return murmur_classifier


def test_xgboost_integration(
    model_name,
    murmur_classifier,
    test_data_directory,
    dbres_output_directory,
    model_binary_pth,
    model_binary_present_pth,
    model_binary_unknown_pth,
    model_type,
    recordings_file,
    bayesian=True,
):

    if os.path.exists(dbres_output_directory):
        recalculated_dbres_output = False
    else:
        recalculated_dbres_output = True
        print(f"DBRes output directory {dbres_output_directory} does not exist. Recalculating DBRes output.")

    # TODO: Add code to load the recordings file.
    _, features_combined = get_murmurs_features(
        model_name,
        test_data_directory,
        recalculated_dbres_output,
        dbres_output_directory,
        model_binary_pth,
        model_binary_present_pth,
        model_binary_unknown_pth,
        model_type=model_type,
        bayesian=bayesian
    )

    murmur_probabilities = murmur_classifier.predict_proba(features_combined)
    murmur_outputs = np.zeros(murmur_probabilities.shape, dtype=np.int_)
    idx = np.argmax(murmur_probabilities, axis=1)
    for i in range(len(murmur_outputs)):
        murmur_outputs[i][idx[i]] = 1

    return murmur_probabilities, murmur_outputs


def calculate_xgboost_integration_scores(
    model_name,
    train_data_directory,
    test_data_directory,
    model_xgb_pth,
    dbres_output_directory,
    model_binary_pth,
    model_binary_present_pth,
    model_binary_unknown_pth,
    output_directory,
    recordings_file,
    use_weights,
    bayesian
):

    if (model_binary_present_pth is not None) and (model_binary_unknown_pth is not None):
        model_type = "murmur"
    elif model_binary_pth is not None:
        if "MurmurBinary" in model_binary_pth:
            model_type = "murmur_binary"
        elif "OutcomeBinary" in model_binary_pth:
            model_type = "outcome_binary"
        else:
            raise Exception("No binary murmur or outcome model was provided.")
    else:
        raise Exception("No model was provided.")

    print(f"--- Using {model_type} model ---")

    # Train
    if train_data_directory is not None:
        print("Training the model.")
        murmur_classifier = train_xgboost_integration(
            model_name,
            train_data_directory,
            os.path.join(dbres_output_directory, "train"),
            model_binary_pth,
            model_binary_present_pth,
            model_binary_unknown_pth,
            model_type=model_type,
            use_weights=use_weights,
            bayesian=bayesian
        )
        # Save the model.
        if "binary" in model_type:
            model_path = ("/").join(model_binary_pth.split("/")[:-1]) + f"/XGB_{model_type}.json"
        else:
            model_path = ("/").join(model_binary_present_pth.split("/")[:-1]) + f"/XGB_{model_type}.json"
        murmur_classifier.save_model(model_path)
        print(f"Model saved to {model_path}")
    else:
        print("No training data was provided. Loading the model.")
        assert model_xgb_pth is not None, "No model or training data were provided."
        murmur_classifier = xgb.XGBClassifier()
        murmur_classifier.load_model(model_xgb_pth)

    # Test
    murmur_probabilities, murmur_outputs = test_xgboost_integration(
        model_name,
        murmur_classifier,
        test_data_directory,
        os.path.join(dbres_output_directory, "test"),
        model_binary_pth,
        model_binary_present_pth,
        model_binary_unknown_pth,
        model_type=model_type,
        recordings_file = recordings_file,
        bayesian=bayesian
    )

    if (model_binary_present_pth is not None) and (model_binary_unknown_pth is not None):
        model_type = "murmur"
    elif model_binary_pth is not None:
        if "MurmurBinary" in model_binary_pth:
            model_type = "murmur_binary"
        elif "OutcomeBinary" in model_binary_pth:
            model_type = "outcome_binary"
        else:
            raise Exception("No binary murmur or outcome model was provided.")
    else:
        raise Exception("No model was provided.")
    print(f"--- Evaluating {model_type} model ---")
    scores = evaluate_model(test_data_directory, murmur_probabilities, murmur_outputs, model_type=model_type, recordings_file = recordings_file, output_directory=output_directory)
    print("---- XGBoost Integration Scores ----")
    print(f"{scores}")
    with open(os.path.join(output_directory, "DBRes_score.npy"), "w") as text_file:
        text_file.write(scores)

    return scores


if __name__ == "__main__":

    parser = argparse.ArgumentParser(prog="XGBoostIntegration")
    parser.add_argument(
        "--model_name",
        type=str,
        help="The ResNet to train. Current options are resnet50 or resnet50dropout.",
        choices=["resnet50", "resnet50dropout"],
        default="resnet50dropout",
    )
    parser.add_argument(
        "--train_data_directory",
        type=str,
        help="The directory of the training data.",
        default=None,
    )
    parser.add_argument(
        "--test_data_directory",
        type=str,
        help="The directory of the test data.",
        default=None,
    )
    parser.add_argument(
        "--model_xgb_pth",
        type=str,
        help="The path of the xgb model. Must be set if no training data are provided.",
        default=None,
    )
    parser.add_argument(
        "--dbres_output_directory",
        type=str,
        help="The directory in which DBRes's output will be saved.",
        default="data/dbres_outputs",
    )
    parser.add_argument(
        "--model_binary_pth",
        type=str,
        help="The path of binary ResNet trained to classify present vs not present.",
        default=None,
    )
    parser.add_argument(
        "--model_binary_present_pth",
        type=str,
        help="The path of binary ResNet trained to classify present vs not present.",
        default=None,
    )
    parser.add_argument(
        "--model_binary_unknown_pth",
        type=str,
        help="The path of binary ResNet trained to classify unknown vs not unknown.",
        default=None,
    )
    parser.add_argument(
        "--output_directory",
        type=str,
        help="The directory in which to save DBRes's output.",
        default="data/dbres_outputs",
    )
    parser.add_argument(
        "--recordings_file",
        type=str,
        help="The path to a recordings file.",
        default="",
    )
    parser.add_argument(
        "--use_weights",
        type=bool,
        help="Whether to use weights in the training data.",
        default=False,
    )
    parser.add_argument(
        '--disable-bayesian',
        dest='bayesian',
        action='store_false',
        default=True,
        help='Disable Bayesian features (default: Bayesian is enabled)'
    )

    args = parser.parse_args()

    print("---------------- Starting xgboost_integration.py for training ----------------")
    print(f"---------------- Using data from {args.train_data_directory}")

    scores = calculate_xgboost_integration_scores(**vars(args))